In [1]:
import numpy as np
import pandas as pd

In [ ]:
class MyDecisionTree:
    def __init__(self, df):
        self.df = df
        self.prediction = None
        self.childs = None
        self.root = None
        self.labelsOfRoot = None
    def __fit__(self):
        if self.df.columns.size != 1:
            if len(np.unique(self.df.iloc[:, -1].to_numpy)) == 1:
                self.prediction = np.unique(self.df.iloc[:, -1].to_numpy)[0]
            else:
                self.root = self.find_best_feature()
                self.labelsOfRoot = np.unique(self.df[self.root].to_numpy())
                self.childs = np.empty((len(self.labelsOfRoot), ), dtype=object)
                for index, label in enumerate(self.labelsOfRoot):
                    self.childs[index] = MyDecisionTree(self.df[self.df[self.root] == label].drop(columns = self.root))
                    self.childs[index].__fit__()
        else:
            self.prediction = self.majority_vote()

    def majority_vote(self):
        pass        

    def find_best_feature(self):
        pass    

In [30]:
df = pd.read_csv('a.csv')
df.drop(columns='a' ,inplace=True)
df.drop(columns='c', inplace=True)
df.iloc[:, -1]

0    2
1    2
2    2
3    2
Name: b, dtype: int64

In [9]:
a = np.array([1, 2, 3])
len(a)

3

In [10]:
a = ['ali', 'reza', 'gholi']
for i, j in enumerate(a):
    print(i, " ", j)

0   ali
1   reza
2   gholi
